In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import csv

#path_train
#path_test 

In [ ]:
def use_data_train(path):

    fichier = open(path)
    L= []
    Y=[]
    T=[]
    for line in fichier :
        L.append(line.split(";"))
    L.pop(0)
    for i in range(len(L)):
        X=[]
        Y.append(int(L[i][1]))
        X.append(int(L[i][2])/3)
        if L[i][3] == 'male':
            X.append(1)
        else :
            X.append(-1)
        if L[i][4]=='':
            X.append(0)
        else: X.append(float(L[i][4])/100)
        X.append(int(L[i][5]))
        X.append(int(L[i][6]))
        X.append(float((L[i][7]))/264)
        T.append(X)

    return np.array(T).T,np.array(Y).T

In [ ]:
def use_data_test(path):

    fichier = open(path)
    L= []
    T=[]
    for line in fichier :
        L.append(line.split(";"))
    L.pop(0)
    for i in range(len(L)):
        X=[]
        X.append(int(L[i][1])/3)
        if L[i][2] == 'male':
            X.append(1)
        else :
            X.append(-1)
        if L[i][3]=='':
            X.append(0)
        else: X.append(float(L[i][3])/100)
        X.append(int(L[i][4]))
        X.append(int(L[i][5]))
        X.append(float((L[i][6]))/264)
        T.append(X)

    return np.array(T).T

In [ ]:
def initialisation(dim):
    param = {}
    C = len(dim)

    for c in range(1, C):
        param['W' + str(c)] = np.random.randn(dim[c], dim[c - 1])
        param['b' + str(c)] = np.random.randn(dim[c], 1)
    return param




In [ ]:
def update(param,grad, learning_rate):
    C = len(param)//2
    for c in range(1,C+1):
        param['W'+str(c)] =  param['W'+str(c)] - learning_rate*grad['dW'+str(c)]
        param['b'+str(c)] =  param['b'+str(c)] - learning_rate*grad['db'+str(c)]
    return param


In [ ]:

def predict(X, param):
    act = forward_prop(X, param)
    C = len(param) // 2
    Af = act['A' + str(C)]
    return Af >= 0.5



In [ ]:

def log_loss(A, y):
    esp = 1e-15
    return (1 / len(y)) * np.sum(-y * np.log(A+esp) - (1 - y) * np.log(1 - A+esp))



In [ ]:

def forward_prop(X, param):
    act = {'A0': X}
    C = len(param) // 2
    for c in range(1, C + 1):
        z = param['W' + str(c)].dot(act['A' + str(c - 1)]) + param['b' + str(c)]
        act['A' + str(c)] = 1 / (1 + np.exp(-z))
    return act



In [ ]:

def back_prop(y, param, act):
    m = y.shape[0]
    C = len(param) // 2

    dZ = act['A' + str(C)] - y
    grad = {}
    for c in reversed(range(1, C + 1)):
        grad['dW' + str(c)] = (1 / m) * np.dot(dZ, act['A' + str(c - 1)].T)
        grad['db' + str(c)] = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
        if c > 1:
            dZ = np.dot(param['W' + str(c)].T, dZ) * act['A' + str(c - 1)] * (1 - act['A' + str(c - 1)])
    return grad

In [ ]:
def neural_network(X, y, hid_layers, learning_rate=0.01, n_iter=10000):
    dim = list(hid_layers)
    dim.insert(0, X.shape[0])
    dim.append(1)
    param = initialisation(dim)
    train_loss = []
    train_acc = []
    for i in tqdm(range(n_iter)):
        act = forward_prop(X, param)
        grad = back_prop(y, param, act)
        param = update(param, grad, learning_rate)

        if i % 10 == 0:
            C = len(param) // 2
            train_loss.append(log_loss(y, act['A' + str(C)]))
            y_pred = predict(X, param)
            current_acc = accuracy_score(y.flatten(), y_pred.flatten())
            train_acc.append(current_acc)

    fig,ax = plt.subplots(nrows=1, ncols=2, figsize=(18,4))
    ax[0].plot(train_loss, label='train loss')
    ax[0].legend()
    ax[1].plot(train_acc, label ='train acc')
    ax[1].legend()
    #visualisation(X,y,param, ax)
    plt.show()

    return param

In [ ]:
X,Y = use_data_train(path_train)
# print(X.shape)
# print(Y.shape)
param = neural_network(X,Y,(6,6,6))

x= use_data_test(path_test)

A = predict(x, param)

def fich(liste) :
    with open ('result.csv','w') as f :
        write = csv.writer(f, delimiter = ',')
        write.writerow(['PassengerId', 'Survived' ])
        for i  in range(len(liste)) :
            if liste[i] == False :
                write.writerow([892 + i, 0])
            else :
                write.writerow([892 + i, 1])

fich(A[0])